# Perceptron wielowarstwowy i metoda wstecznej propagacji błędów

In [1]:
import numpy as np

In [2]:
class NeuralNetwork_FF:

    def __init__(self, input_size, output_size, show = False):

        np.random.seed(42)

        self.show = show

        if self.show: print("Parametry tworzonej sieci:","\n")

        self.input_size = input_size
        self.output_size = output_size

        if self.show:
            print("Liczba neuronów wejściowych: ", input_size)
            print("Liczba neuronów wyjściowych: ", output_size)

        self.weights_input_output = np.random.randn(self.input_size, self.output_size)
        self.bias_output = np.random.randn(1, self.output_size)

        if self.show:
            print("\nWagi neuronów wyjściowych:")
            print(self.weights_input_output)
            print("\nWartości progowe neuronów wyjściowych:")
            print(self.bias_output)
            print("\n-----------------------------\n")

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
 
    def feedforward(self, X):

        if self.show: print("Wektor wejściowy: ",X)
        
        self.output_activation = np.dot(X, self.weights_input_output) + self.bias_output
        if self.show: print("Zagregowane wartości wejściowe dla neuronów wyjściowych: ", self.output_activation)


        self.predicted_output = self.sigmoid(self.output_activation)
        if self.show: print("Wartości wyjściowe neuronów ukrytych: ", self.predicted_output)

        return self.predicted_output

In [3]:
nn = NeuralNetwork_FF(input_size=3,output_size=2, show = True)

X = np.array([2,5, 4])

nn.feedforward(X)

Parametry tworzonej sieci: 

Liczba neuronów wejściowych:  3
Liczba neuronów wyjściowych:  2

Wagi neuronów wyjściowych:
[[ 0.49671415 -0.1382643 ]
 [ 0.64768854  1.52302986]
 [-0.23415337 -0.23413696]]

Wartości progowe neuronów wyjściowych:
[[1.57921282 0.76743473]]

-----------------------------

Wektor wejściowy:  [2 5 4]
Zagregowane wartości wejściowe dla neuronów wyjściowych:  [[4.87447031 7.16950758]]
Wartości wyjściowe neuronów ukrytych:  [[0.99241877 0.99923089]]


array([[0.99241877, 0.99923089]])

## Funkcja sigmoidalna i jej pochodna

Funkcja sigmoidalna:

$$
f(x) = \frac{1}{1+e^{-x}}
$$

Pochodna funkcji sigmoidalnej:

$$
f'(x)=-\frac{1}{(1+e^{-x})^2} (-e^{-x}) = \frac{1}{1+e^{-x}} \times \frac{(1 + e^{-x})-1}{1 + e^{-x}} = f(x)(1-f(x))
$$



## Modyfikacja wag

Błąd sieci:

$$
E = \frac{1}{2} \sum_{m=1}^N (y_m - d_m)^2
$$

gdzie $y_i$ jest wartością wyjściową, zaś $d_i$ wartością rzeczywistą.

Wagi modyfikujemy metodą gradientową.

$$
\frac{\partial E}{\partial w_{ij}} = \frac{\partial E}{\partial y_j} \times \frac{\partial y_j}{\partial a_j} \times \frac{\partial a_j}{\partial w_{ij}}
$$

$$
\frac{\partial E}{\partial y_j} = y_j - d_j
$$

$$
\frac{\partial y_j}{\partial a_j} = y_j(1-y_j)
$$

$$
\frac{\partial a_j}{\partial w_{ij}} = x_i
$$

Ostatecznie uzyskujemy:

$$
\frac{\partial E}{\partial w_{ij}} = (y_j - d_j) y_j(1-y_j) x_i
$$

Reguła uczenia:

$$
w_{ij}(t+1) = w_{ij}(t) - \eta \frac{\partial E}{\partial w_{ij}}
$$

In [4]:
class NeuralNetwork(NeuralNetwork_FF):
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def backward(self, X, y, learning_rate):
        output_error = y - self.predicted_output
        output_delta = output_error * self.sigmoid_derivative(self.predicted_output)

        self.weights_input_output += np.dot(X.T, output_delta) * learning_rate
        self.bias_output += np.sum(output_delta, axis=0,keepdims=True) * learning_rate
        
    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            output = self.feedforward(X)
            self.backward(X, y, learning_rate)
            if epoch % 4000 == 0:
                loss = np.mean(np.square(y - output))
                print(f"Epoka {epoch}, Loss:{loss}")

In [5]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [1]])

nn = NeuralNetwork(input_size=2, output_size=1)
nn.train(X, y, epochs=10000, learning_rate=0.2)

output = nn.feedforward(X)
print("\nWartości wyjściowe po zakończeniu uczenia:")
print(output)

Epoka 0, Loss:0.17546525262263657
Epoka 4000, Loss:0.001728153913929981
Epoka 8000, Loss:0.0008124443022637284

Wartości wyjściowe po zakończeniu uczenia:
[[0.03784769]
 [0.9762643 ]
 [0.97626469]
 [0.99997675]]
